<a href="https://colab.research.google.com/github/purang2/prompting-nlp/blob/main/KoGPT_nsmc_prompt%233.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **[ Produced by ]**

[Eunchan Lee of KNU-BrainAI](https://github.com/purang2)


**[ 목차 ]**


---



### Requirements 설치 

In [1]:

import gc, torch

gc.collect()
torch.cuda.empty_cache()

In [2]:
!pip install transformers 
!pip install accelerate
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 20.6 MB/s 
     |████████████████████████████████| 163 kB 85.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 143 kB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 87.7 MB/s 
     |████████████████████████████████| 115 kB 92.9 MB/s 
     |████████████████████████████████| 127 kB 96.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

## Model Loading

Huggingface/transformers 패키지를 통해 토크나이저와 모델을 load 

Model size: 11.5 GB (큰 것 같다 -> 16GB 권장, 고용량의 RAM 필요)
- KoGPT6B-ryan1.5b는 32GB GPU RAM을 최소로 권장 
- KoGPT6B-ryan1.5b-float16는 16GB GPU RAM을 권장


-> it takes about 5 min

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16', # or float32 version: revision=KoGPT6B-ryan1.5b
    bos_token = '[BOS]', eos_token='[EOS]', unk_token = '[UNK]', pad_token='[PAD]', mask_token = '[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',# or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

_  = model.eval()

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.3G [00:00<?, ?B/s]

### KLUE-MRC

In [ ]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 17554
    })
    validation: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 5841
    })
})

In [ ]:
klue_mrc = dataset['train']
klue_mrc

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 17554
})

In [ ]:
klue_mrc[1]['context']

'부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다. 과학기술정보통신부가 주관하는 ‘지역SW서비스사업화 지원사업’은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다. 올해부터 2개년으로 진행되며, 국비와 시비, 민자 등 2년간 약 37억원의 예산이 투입된다. 앞서 진흥원은 부산의 미래 먹거리산업인 스마트해양, 지능형기계, 지능정보서비스 분야로 사전 수요조사를 진행했고, 평가를 통해 선정된 5개 과제를 공모사업에 신청했다. 그 결과 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀다. 당 사업은 전국 진흥기관을 대상으로 공모를 시작해, 총 17개 지역에서 42개 과제가 선정되었으며, 4개 과제가 선정된 곳은 부산과 강원지역 뿐이다. 금번 선정된 과제들은 ‘인공지능융합센서와 서보 이송 로봇을 이용한 전단보강재의 자동용접시스템 개발’ 등 총 4개 과제다. 부산시가 지원하고, 부산정보산업진흥원과 지역기업, 대학, 연구소 등이 컨소시엄을 구성하여 기술개발 및 사업화 지원을 추진한다. 2개의 Track으로 구분되는 이번사업은 Track 1(SW중소기업)에서 ㈜에이아이플랫폼, 엔컴(주), Track 2(스타트업)에서는 ㈜토즈, 삼보테크놀로지를 지원한다. ○ ‘Track 1‘의 (주)에이아이플랫폼이 주관기업으로 진행하는 <인공지능 기반 망막 내 아밀로이드 플라크 영상 분석을 통한 치매조기진단 플랫폼 상용화>는 치매 확진의 원인이 되는 중요 단백질(아밀로이드 플라크)을 자체개발 관측장비로 진단한다. 이를 통해 치매를 조기 발견하여, 각종 경제적 비용과 치료 및 예방 등 사회적 문제를 해 결하고 시민들이 쉽게 접근 가능한 실효성 있는 치매관리체계 개발을 목표로 한다. ○ 엔컴(주)이 주관기업으로 참여하는 <AI영상분석 기반 가공철근 생산성 향상 시스템 기술개발 및 사업화>는 산업안전,

In [ ]:

klue_mrc[1]['source']

'acrofan'

In [ ]:
#len(klue_mrc[1]['context'])

klue_mrc[1]['question']





'성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?'

In [ ]:

klue_mrc[1]['answers']


{'answer_start': [18, 21, 1673], 'text': ['원장 이인숙', '이인숙', '이인숙']}

In [ ]:
ex = klue_mrc['title']

len(ex)

17554

In [ ]:
ex = list(set(ex))


len(ex)



12169

### Experimental Setup 


  논문 [Onthe Effect of Pretraining Corpora on In-context Learning by a Large-scale Language Model]의 실험 섹션인 $3.4를 참고한다.

  - Reading Comprehension 태스크인 KorQuAD에서 4 Shot Learning을 적용하였다.




In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [ ]:
dataset['validation']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5774
})

In [ ]:
data = dataset['train']

In [ ]:
data[0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]}}

In [ ]:
data[50]['answers']['text'][0]

'이북 구간'

In [ ]:
data['context'][52803]   # 52800 ~ 52803

'다음은 미국의 여배우 제시카 채스테인의 작품 목록이다. 줄리아드 스쿨에서 졸업반 학생이었던 그녀는 TV 프로듀서인 존 웰스의 눈에 띄어 계약에 서명했다. 2004년부터 2010년까지 《ER》, 《베로니카 마스》, 《로 앤 오더: 배심원에 의한 재판》등 여러 TV 프로그램에서 게스트 역할로 출연했다. 그녀는 2004년 미셸 윌리엄스와 《벚꽃 동산》, 2006년 알 파치노와 《살로메》와 같은 무대 연극에도 출연했다. 2008년 채스테인은 E. L.닥터로의 단편 소설 Jolene: A Life를 각색한 댄 아일랜드 감독의 드라마 영화 《조렌느》에서 타이틀 롤을 맡으며 영화 데뷔를 하였다. 그녀는 비판적인 평가를 받은 미스테리 스릴러 영화 《스틸》에서 작은 역할을했으며, 그 후 그녀는 《언피니시드》(2010)에서 헬렌 미렌이 분한 레이철 싱어 역할의 더 어린 버전을 연기했다.'

### K=50 Few-shot TC [NSMC]

In [5]:
from datasets import load_dataset

dataset = load_dataset("nsmc")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [18]:
data_document = dataset['train'][959:959+25]['document']
data_document

['영화사에 남을 작품. 실험영화의 완성본',
 '자식을 그렇게 때리냐?',
 '이게 대체 무슨 내용이야;;',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ옛다 1점이나 처먹어랔ㅋㅋㅋㅋㅋㅋㅋㅋ이게 7점급?이건 1점짜리. 내용도 결말도. 살다 살다 뭐 이딴...',
 '감동적이였고 좋았습니다',
 '초반에 개재미없었는데 볼수록 점점 재밌어짐! 외계인이 내가 알던 에일리언에 나오던 외계인이 아니야....',
 '쇼핑백 잊혀지지 않는다....',
 '의도는 좋지만 너무 몰입이 안되는걸 어떡해',
 '유치하지만 가볍게 볼만함',
 '화려한 색채때문에 눈이 아프지만 그 나름대로 화려연예계여자욕망에대해 표현해냈던거같다 보는내내 진짜 리리코심정가진 연옌들도 있을거같고..나를한번도보지못하고알지못하는사람들이날어떻게사랑하냐그런대사 나왔을때 소름돋더라 연예인들은 많은사랑받으면서도 참 허전할듯',
 '본지 수OO이 지나도 아직도 생생함. 고양이 귀신의 복수가 진짜 무서웠지',
 '영상이 대학교 졸업작품같아서 그렇지 꽤나 많은걸 담아놓은 영화임에는 분명하다.',
 '최강 전차부대 ..,,..',
 'ㅇㅇ',
 '진짜 어이없는 소재라고 생각하며 보는데 개 웃겨.... 남자로서는 솔직히 조금은 부러울수도 있는',
 'tv 전기세가 아까웠다!!!',
 '초등학교때 선생님이 보여주신 영화 시간이 흘러서 기억나는건 에이즈에 걸렸던 남자아이와 운동화였고 그때 봤던 영화가 뭐였을까? 궁금했었는데 그 영화가 바로 굿바이마이프렌드였다 12년전이나 12년후나 명작은 시간이 흘러도 명작이다',
 '넘 재밌었어요/ㅅ/ㅎㅎㅎ',
 '설리 94년생인데 93년도 영화에 어떻게 나오져?',
 '샷의 연결이 아닌 충돌만큼이나 거룩한 제목 또한 인상적이구려',
 'ㅋㅋㅋ 재미없음',
 '오오 난 여태까지 맷데이먼의 최고의영화는 본 얼티메이텀,라이언일병구하기 뿐인줄알았음ㅠ 굿윌헌팅도쩐당',
 '이야기 자체가 흥미를 끌지 못하는군',
 '날 로그인하게 만들다니.....청춘이여 영원하라....',
 '현정언니 짱이죠 완전 잼잇어요 ㅎ'

In [19]:
data_label =dataset['train'][959:959+25]['label']

lbl = ['긍정','부정']
data_label

[1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]

prompt setting

In [20]:
prompt = ''
#i=0

#nsmc prompt #1
'''
for i in range(len(data_document)):
  prompt+=f'{data_document[i]}({lbl[data_label[i]]})\n'
'''

#nsmc prompt #2
for i in range(len(data_document)):
  prompt+=f'{{{data_document[i]}}}\n 이 리뷰는 {{긍정,부정}}중 무엇?\n{{{lbl[data_label[i]]}}}\n\n'


print(prompt)

{영화사에 남을 작품. 실험영화의 완성본}
 이 리뷰는 {긍정,부정}중 무엇?
{부정}

{자식을 그렇게 때리냐?}
 이 리뷰는 {긍정,부정}중 무엇?
{긍정}

{이게 대체 무슨 내용이야;;}
 이 리뷰는 {긍정,부정}중 무엇?
{긍정}

{ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ옛다 1점이나 처먹어랔ㅋㅋㅋㅋㅋㅋㅋㅋ이게 7점급?이건 1점짜리. 내용도 결말도. 살다 살다 뭐 이딴...}
 이 리뷰는 {긍정,부정}중 무엇?
{긍정}

{감동적이였고 좋았습니다}
 이 리뷰는 {긍정,부정}중 무엇?
{부정}

{초반에 개재미없었는데 볼수록 점점 재밌어짐! 외계인이 내가 알던 에일리언에 나오던 외계인이 아니야....}
 이 리뷰는 {긍정,부정}중 무엇?
{부정}

{쇼핑백 잊혀지지 않는다....}
 이 리뷰는 {긍정,부정}중 무엇?
{긍정}

{의도는 좋지만 너무 몰입이 안되는걸 어떡해}
 이 리뷰는 {긍정,부정}중 무엇?
{긍정}

{유치하지만 가볍게 볼만함}
 이 리뷰는 {긍정,부정}중 무엇?
{긍정}

{화려한 색채때문에 눈이 아프지만 그 나름대로 화려연예계여자욕망에대해 표현해냈던거같다 보는내내 진짜 리리코심정가진 연옌들도 있을거같고..나를한번도보지못하고알지못하는사람들이날어떻게사랑하냐그런대사 나왔을때 소름돋더라 연예인들은 많은사랑받으면서도 참 허전할듯}
 이 리뷰는 {긍정,부정}중 무엇?
{부정}

{본지 수OO이 지나도 아직도 생생함. 고양이 귀신의 복수가 진짜 무서웠지}
 이 리뷰는 {긍정,부정}중 무엇?
{부정}

{영상이 대학교 졸업작품같아서 그렇지 꽤나 많은걸 담아놓은 영화임에는 분명하다.}
 이 리뷰는 {긍정,부정}중 무엇?
{부정}

{최강 전차부대 ..,,..}
 이 리뷰는 {긍정,부정}중 무엇?
{긍정}

{ㅇㅇ}
 이 리뷰는 {긍정,부정}중 무엇?
{긍정}

{진짜 어이없는 소재라고 생각하며 보는데 개 웃겨.... 남자로서는 솔직히 조금은 부러울수도 있는}
 이 리뷰는 {긍정,부정}중 무엇?
{부정}

{tv 전기세가 아까웠다!!!}
 이 리뷰는 {긍정,부정}

In [21]:
len(prompt)

1602

In [28]:
eval_data = dataset['test']

eval_data

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 50000
})

In [29]:
len(eval_data)

50000

In [30]:
tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
len(tokens[0])

962

### RUN Inference

In [27]:

  gc.collect()
  torch.cuda.empty_cache()

In [31]:
eval_data = dataset['test']

answer_set = [] #answer set 


#prompt 

'''
#1: 아래 지문을 읽고 물음에 답을 유추하시오. 
#2: 질의응답 태스크 수행: 
#3: {맥락} \n {질문}에 대해 답이 뭔지 아니? 

'''
import gc

#for i in range(len(eval_data)):
for i in range(1000):
  
  gc.collect()
  torch.cuda.empty_cache()
  #index = 2007 + i*20
  index = i
  #prompt_with_query = prompt + '질의응답 태스크 수행:\n\n' + f"맥락: {eval_data[index]['context']} \n 질문: {eval_data[index]['question']} \n 대답:"
  #prompt_with_query = prompt +  f"맥락: {eval_data[index]['context']} \n 질문: {eval_data[index]['question']} {promptz} \n 대답:"
  prompt_with_query = prompt +  f"{{{eval_data[i]['document']}}}\n 이 리뷰는 {{긍정,부정}}중 무엇?\n"
#f'리뷰: {data_document[i]}\n 감정:{lbl[data_label[i]]}\n'
#f'{{{data_document[i]}}}\n 이 리뷰는 {{긍정,부정}}중 무엇?\n{{{lbl[data_label[i]]}}}\n\n'

  with torch.no_grad():
    tokens = tokenizer.encode(prompt_with_query, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=len(tokens[0])+6)
    generated = tokenizer.batch_decode(gen_tokens)[0]

  #print(generated[len(prompt):])
  #answer_set.append(generated[generated.rfind("대답:")+4:])
  answer_set.append(generated[-6:])
  if i % 10 == 0:
    print(i)
  #print(f" Answer: {eval_data[index]['answers']['text']} \n\n")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [32]:
answer_set[270][-5:]


'정}\n\n{'

In [33]:
answer_set[0].find('긍정')

0

In [34]:
p_len, n_len =0, 0
for i in range(len(answer_set)):
  t = answer_set[i]

  if t.find('긍정') != -1: p_len+=1
  elif t.find('부정') != -1: n_len+=1

print(p_len)
print(n_len)
print('Else: ',len(answer_set)-p_len-n_len)

435
563
Else:  2


In [35]:
answer_set[70:120]

['부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '긍정}\n\n{',
 '긍정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{',
 '부정}\n\n{',
 '긍정}\n\n{']

In [36]:
import pickle

filePath = 'drive/MyDrive/졸논_nsmc_prompt#3.txt'
#testList = ['123', '456', 'abc', 'def']
with open(filePath, 'wb') as lf:
    pickle.dump(answer_set, lf)

In [ ]:
import pickle


filePath1 = 'drive/MyDrive/졸논_nsmc_prompt1.txt'

with open(filePath1, 'rb') as lf:
    readList1 = pickle.load(lf)
    print(readList1)
    # ['123', '456', 'abc', 'def']


p_len, n_len =0, 0
for i in range(len(readList1)):
  t = readList1[i]

  if t.find('긍정') != -1: p_len+=1
  elif t.find('부정') != -1: n_len+=1

print(p_len)
print(n_len)
print('Else: ',len(readList1)-p_len-n_len)

['(부정)\n', '라는게 이', '?(긍정)', '.(부정)', '(긍정)\n', '.(긍정)', '(부정)\n', '.(부정)', '.(부정)', '(긍정)\n', '(긍정)\n', ' 영화 다', '(긍정)\n', '다도 배우', '(긍정)\n', '(부정)\n', '!(긍정)', '(부정)\n', '!(부정)', '(긍정)\n', '금의 공포', '(긍정)\n', '들 연기도', '(긍정)\n', '(긍정)\n', ', 그 놈', '?(긍정)', '.(부정)', '(부정)\n', '화가 끝난', '!(긍정)', '(긍정)\n', '(부정)\n', '말,진짜,', '(부정)\n', '(부정)\n', '걸그랬어요', '(부정)\n', '(부정)\n', '(긍정)\n', ' 기억안나', '(부정)\n', '미디야?(', '국이 을임', '는 영화랑', '(부정)\n', '.(부정)', '지 흘리죠', '(긍정)\n', '(긍정)\n', ' 감독해도', '지루하다(', '(부정)\n', '013년.', '(긍정)\n', '(긍정)\n', '.(긍정)', '야!!!(', '!(긍정)', '아직도 그', '(부정)\n', ' 이런거?', '서 아빠가', '(부정)\n', '여..ᄒᄒ', '(부정)\n', '(부정)\n', '(긍정)\n', ' 보고싶음', '(긍정)\n', '신의 죄를', '우들 연기', '(긍정)\n', '(부정)\n', ' 관객모독', '(부정)\n', '들 연기도', '(긍정)\n', '~(부정)', '(긍정)\n', '아무리 좋', '.(긍정)', '.(긍정)', '^;(부정', '(부정)\n', '(부정)\n', '(긍정)\n', '(부정)\n', '(긍정)\n', ' 보지마셈', '(긍정)\n', '했는데..', '.(부정)', '\n재밌었다', '냥 저냥한', '(긍정)\n', '(부정)\n', '(부정)\n', '이나.ᄏᄏ', '(긍정)\n', '(부정)\n', '없다(긍정', ' 좋은영화', '션의 비중', '의 나에게', '.(

## RUN If your RAM is down.. 

In [ ]:

import gc
gc.collect()
torch.cuda.empty_cache()



# Evaluation

## Inference Score with F1-as metric

- answer #1: prompt[0], with golden label, with k=4 
- answer #2: prompt[0], with random label, with k=4 
- answer #3: zeroshot ,no prompt, with k=4 
- answer #4: prompt[1], with golden label ,with k=4 
- answer #5: prompt[2], with golden label, with k=4 




Result #1

올바른 데이터가 중요한가? 
포맷만 맞으면 된다? 

비교: answer #1, #2, #3




Result #2

prompt에 따라 어떻게 변하는가?



비교: result #1, #4, #5 






In [ ]:
import pickle


filePath1 = './answer_set_prompt1.txt'

with open(filePath1, 'rb') as lf:
    readList1 = pickle.load(lf)
    print(readList1)
    # ['123', '456', 'abc', 'def']




In [ ]:
filePath2 = './answer_set_prompt2.txt'


with open(filePath2, 'rb') as lf:
    readList2 = pickle.load(lf)
    print(readList2)
    # ['123', '456', 'abc', 'def']

In [ ]:
filePath = './answer_set_prompt3_nodemo.txt'


with open(filePath, 'rb') as lf:
    readList3 = pickle.load(lf)
    print(readList3)
    # ['123', '456', 'abc', 'def']

In [ ]:
filePath = './answer_set_prompt4_new_prompt.txt'


with open(filePath, 'rb') as lf:
    readList4 = pickle.load(lf)
    print(readList4)
    # ['123', '456', 'abc', 'def']

In [ ]:
filePath = './answer_set_prompt5_new_prompt.txt'

with open(filePath, 'rb') as lf2:
    readList5 = pickle.load(lf2)
    print(readList5)
    # ['123', '456', 'abc', 'def']

In [ ]:
!pip install datasets

In [ ]:
'''
#1 : readList1
#2 : readList2
#3 : readList3
#4 : readList4
#5 : readList5

'''
from datasets import load_dataset

#dataset = load_dataset("klue", "mrc")


dataset = load_dataset("squad_kor_v1")

answer = dataset['validation']['answers']



for i in range(len(answer)):
  try:
    answer[i] = answer[i]['text'][0]
  except:
    print(i)


#answer


In [ ]:
#readList1

def preprocessing(text):
  if text.find('[끝]:') != -1:
    text = text[:text.find('[끝]:')]
  if text.find('[끝:') != -1:
    text = text[:text.find('[끝:')]
  if text.find('[끝]') != -1:
    text = text[:text.find('[끝]')]
  if text.find('[끝') != -1:
    text = text[:text.find('[끝')]
  if text.find('질문:') != -1:
    text = text[:text.find('질문:')]
  
  return text





for i in range(len(readList1)):
  readList1[i] = preprocessing(readList1[i])

for i in range(len(readList2)):
  readList2[i] = preprocessing(readList2[i])

for i in range(len(readList3)):
  readList3[i] = preprocessing(readList3[i])

for i in range(len(readList4)):
  readList4[i] = preprocessing(readList4[i])

for i in range(len(readList5)):
  readList5[i] = preprocessing(readList5[i])




print(readList5)


In [ ]:
len(answer)

In [ ]:
len(readList1)

In [ ]:
len(readList2)

In [ ]:
'''
#1 : readList1
#2 : readList2
#3 : readList3
#4 : readList4
#5 : readList5


answer 
'''


f1_1 = 0
f1_2 = 0
f1_3 = 0
f1_4 = 0
f1_5 = 0


for i in range(len(answer)):
  f1_1 += f1_score(answer[i],readList1[i])
  f1_2 += f1_score(answer[i],readList2[i])
  f1_3 += f1_score(answer[i],readList3[i])
  f1_4 += f1_score(answer[i],readList4[i])
  f1_5 += f1_score(answer[i],readList5[i])



print(f1_1)
print(f1_2)
print(f1_3)
print(f1_4)
print(f1_5)





print(f1_1*100/len(answer))
print(f1_2*100/len(answer))
print(f1_3*100/len(answer))
print(f1_4*100/len(answer))
print(f1_5*100/len(answer))




In [ ]:
'''
#1 : readList1
#2 : readList2
#3 : readList3
#4 : readList4
#5 : readList5


answer 
'''


em_1 = 0
em_2 = 0
em_3 = 0
em_4 = 0
em_5 = 0


for i in range(len(answer)):
  em_1 += em_score(answer[i],readList1[i])
  em_2 += em_score(answer[i],readList2[i])
  em_3 += em_score(answer[i],readList3[i])
  em_4 += em_score(answer[i],readList4[i])
  em_5 += em_score(answer[i],readList5[i])



print(em_1)
print(em_2)
print(em_3)
print(em_4)
print(em_5)





print(em_1*100/len(answer))
print(em_2*100/len(answer))
print(em_3*100/len(answer))
print(em_4*100/len(answer))
print(em_5*100/len(answer))




In [ ]:
'''
#1 : readList1
#2 : readList2
#3 : readList3
#4 : readList4
#5 : readList5


answer 


Archive: 자체적인 메트릭으로 score를 내린 결과 -> 논문에는 f1-score/exact-match를 메트릭으로 하는 것이 훨씬 적절함 


f1_1 = len(answer)  
f1_2 = len(answer)
f1_3 = len(answer)
f1_4 = len(answer)
f1_5 = len(answer) 


j = '안중EEWEz'
j.lower() #안중eewez




#Lower, delete all spaces,
for i in range(len(answer)):
  readList1[i] =readList1[i].lower().strip()
  readList2[i] =readList2[i].lower().strip()
  readList3[i] =readList3[i].lower().strip()
  readList4[i] =readList4[i].lower().strip()
  readList5[i] =readList5[i].lower().strip()



for i in range(len(answer)):
  if readList1[i].find(answer[i]) == -1 and answer[i].find(readList1[i]) == -1: f1_1 = f1_1 -1 
  if readList2[i].find(answer[i]) == -1 and answer[i].find(readList2[i]) == -1: f1_2 = f1_2 -1 
  if readList3[i].find(answer[i]) == -1 and answer[i].find(readList3[i]) == -1: f1_3 = f1_3 -1 
  if readList4[i].find(answer[i]) == -1 and answer[i].find(readList4[i]) == -1: f1_4 = f1_4 -1 
  if readList5[i].find(answer[i]) == -1 and answer[i].find(readList5[i]) == -1: f1_5 = f1_5 -1 



print(f1_1)
print(f1_2)
print(f1_3)
print(f1_4)
print(f1_5)



print(f1_1*100/len(answer))
print(f1_2*100/len(answer))
print(f1_3*100/len(answer))
print(f1_4*100/len(answer))
print(f1_5*100/len(answer))



'''




In [ ]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("\n", " ", text)
        text = re.sub("\[", " ", text)
        text = re.sub("\]", " ", text)      
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
   
    #F1 by character
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)
        
    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)   
        
    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)
    
    return f1




def em_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


In [ ]:
x = f1_score('보수적 근본주의 계열 개신교','보수적 근본주의계열의 개신교')

In [ ]:
x

In [ ]:
x = f1_score('\'1869년 메이지 2년 3월 25일\'','1879년 메이지 2년 3월 25일')

In [ ]:
x